In [6]:
from __future__ import print_function, division
import warnings
# warnings.filterwarnings("ignore")
import os
# os.system("pip install torchvision")
import os.path
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import dlib
# import argparse
from os import listdir
from os.path import isfile, join
from tqdm import tqdm

In [11]:
def rect_to_bb(rect):
	# take a bounding predicted by dlib and convert it
	# to the format (x, y, w, h) as we would normally do
	# with OpenCV
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
	# return a tuple of (x, y, w, h)
	return (x, y, w, h)



def predict_gender(save_prediction_at, imgs_path = 'cropped_faces/', score_precision=3, append=False):
    img_paths = [os.path.join(imgs_path, x) for x in os.listdir(imgs_path)]
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Device in use: {device}")

    model_fair_7 = torchvision.models.resnet34(weights="DEFAULT")
    model_fair_7.fc = nn.Linear(model_fair_7.fc.in_features, 18)
    model_fair_7.load_state_dict(torch.load('fair_face_models/fairface_alldata_7race_20191111.pt'))
    model_fair_7 = model_fair_7.to(device)
    model_fair_7.eval()

    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    # img pth of face images
    face_names = []
    # list within a list. Each sublist contains scores for all races. Take max for predicted race
    gender_scores_fair = []
    gender_preds_fair = []

    print("Predicting...")
    for index, img_path in enumerate(tqdm(img_paths)):
        # if index % 1000 == 0:
        #     print(f"Predicting... {index}/{len(img_paths)}")

        img_name = img_path.split("/")[1]

        face_names.append(img_name)
        image = dlib.load_rgb_image(img_path)
        image = trans(image)
        image = image.view(1, 3, 224, 224)  # reshape image to match model dimensions (1 batch size)
        image = image.to(device)

        # fair
        outputs = model_fair_7(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        gender_outputs = outputs[7:9]
        gender_score = np.round(np.exp(gender_outputs) / np.sum(np.exp(gender_outputs)), score_precision)

        gender_pred = np.argmax(gender_score)
        gender_scores_fair.append(gender_score)
        gender_preds_fair.append(gender_pred)


    result = pd.DataFrame([face_names,
                           gender_preds_fair,
                           gender_scores_fair,
                           ]).T
    result.columns = ['face_name_align',
                      'gender_preds_fair',
                      'gender_scores_fair'
                      ]


    # gender
    result.loc[result['gender_preds_fair'] == 0, 'gender'] = 'man'
    result.loc[result['gender_preds_fair'] == 1, 'gender'] = 'woman'

    result = result.sort_values("face_name_align")

    result[['face_name_align',
            'gender', 
            'gender_scores_fair'
            ]].to_csv(save_prediction_at, index=False, mode="a", header=not append)

    print(f"Saved results at '{save_prediction_at}'")


def prepare_prediction(detections_dir, prediction_dir):
    
    # detections should already be prepared
    if not os.path.exists(detections_dir):
        raise Exception(f"There are no detections to predict at '{detections_dir}'!")
    else:
        print(f"Detected faces found at '{detections_dir}'.")
    

    # ensure directory for prediction exists
    if not os.path.exists(prediction_dir):
        os.makedirs(prediction_dir)


def get_image_number(detections_dir):
    detected_faces = [f for f in listdir(detections_dir)]
    return detected_faces[0].split(".")[0].split("_")[0]


dlib.DLIB_USE_CUDA = True

In [3]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--csv', dest='input_csv', action='store',
#                     help='csv file of image path where col name for image path is "img_path')
# args = parser.parse_args()
# imgs = pd.read_csv(args.input_csv)['img_path']
# imgs = pd.read_csv(input_csv)['img_path']

In [2]:
# pick directories
# os.system("CUDA_LAUNCH_BLOCKING=1")
detections_dir = "detected_faces_CelebA"
prediction_dir = "outputs"

# do you want to append the predictions to former predictions?
append = True

# prepare the prediction
prepare_prediction(detections_dir, prediction_dir)

# create prediction file name
output_csv = f"{prediction_dir}/test.csv"

# do the prediction
predict_gender(save_prediction_at=output_csv, imgs_path=detections_dir, append=append)